In [1]:
import os
import json
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [4]:

json_path = 'C:\\Rohit\\Projects\\Image Data Set\\Flickr30k\\annotations.json' 

with open(json_path, 'r') as f:
    data = json.load(f)

print(f"Length : {len(data.items())}")
for image_name, captions in data.items():
    print(f"Image: {image_name}")
    print(f"Captions: {captions}")
    print("\n")

KeyError: 'annotations'